In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [19]:
#https://www.kaggle.com/johnfarrell/breaking-lb-fresh-start-with-lag-selection/output
train_base=pd.read_csv('train.csv')
test_base=pd.read_csv('test.csv')

train_leak=pd.read_csv('train_leak.csv')
test_leak=pd.read_csv('test_leak.csv')

trainleak = train_leak['compiled_leak'].values
trainlogleak = np.log1p(train_leak['compiled_leak'].values)
testleak = test_leak['compiled_leak'].values
testlogleak = np.log1p(test_leak['compiled_leak'].values)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in log1p
  # This is added back by InteractiveShellApp.init_path()


In [20]:
def drop_vars(df):
    tmp=df.shape[1]
    df = df[df.columns[[True]+list((df.var()!=0))]]
    print('0 var:',tmp-df.shape[1])
    
    corr_matrix = df[df.columns[2:]].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    tmp=df.shape[1]
    df=df.drop(to_drop, axis=1)
    print('Corr>0.95:',tmp-df.shape[1])
    
    corrs = dict()
    for i in range(df.shape[1]-2):
        corrs[df.columns[2+i]] = np.corrcoef(df['target'],df[df.columns[2+i]])[0,1]
    s = [k for k in corrs if abs(corrs[k])<0.1]
    tmp=df.shape[1]
    df=df.drop(s, axis=1)
    print('Corr Target <0.1:',tmp-df.shape[1])
    
    return df
tmp=drop_vars(train_base)

0 var: 256
Corr>0.95: 139
Corr Target <0.1: 4206


In [21]:
train = tmp

train['leak'] = trainleak
train['log_leak'] = trainlogleak
test['leak'] = testleak
test['log_leak'] = testlogleak

#https://www.kaggle.com/ogrellier/feature-scoring-vs-zeros/notebook
features = [f for f in train if f not in ['ID', 'leak', 'log_leak', 'target']]
train.replace(0, np.nan, inplace=True)
train['log_of_mean'] = np.log1p(train[features].replace(0, np.nan).mean(axis=1))
train['mean_of_log'] = np.log1p(train[features]).replace(0, np.nan).mean(axis=1)
train['log_of_median'] = np.log1p(train[features].replace(0, np.nan).median(axis=1))
train['nnans'] = train[features].isnull().sum(axis=1)
train['sum'] = np.log1p(train[features].sum(axis=1))
train['std'] = train[features].std(axis=1)
train['kurtosis'] = train[features].kurtosis(axis=1)

test.replace(0, np.nan, inplace=True)
test['log_of_mean'] = np.log1p(test[features].replace(0, np.nan).mean(axis=1))
test['mean_of_log'] = np.log1p(test[features]).replace(0, np.nan).mean(axis=1)
test['log_of_median'] = np.log1p(test[features].replace(0, np.nan).median(axis=1))
test['nnans'] = test[features].isnull().sum(axis=1)
test['sum'] = np.log1p(test[features].sum(axis=1))
test['std'] = test[features].std(axis=1)
test['kurtosis'] = test[features].kurtosis(axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log1p
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in log1p
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log1p
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log1p
  


In [22]:
print(train.shape)
print(test.shape)
train.head()

(4459, 401)
(49342, 5001)


,ID,target,20aa07010,4681de4fd,adf119b9a,d966ac62c,87ffda550,822e49b95,64534cc93,251d1aa17,...,5831f4c76,leak,log_leak,log_of_mean,mean_of_log,log_of_median,nnans,sum,std,kurtosis
0,000d6aaf2,38000000.0,NaN,NaN,NaN,NaN,1300000.0,NaN,NaN,4333333.34,...,NaN,38000000.0,17.453097,15.457080,14.615507,14.508658,339,19.388906,8.093122e+06,7.942924
1,000fbd867,600000.0,2200000.0,NaN,NaN,NaN,NaN,NaN,NaN,2000000.00,...,NaN,600000.0,13.304687,14.986185,14.717394,14.508658,350,18.675065,3.496137e+06,8.679952
2,0027d6b71,10000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,15.586217,15.240200,15.564711,382,17.665658,4.257347e+06,-0.820324
3,0028cbf45,2000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2000000.0,14.508658,15.019484,14.874862,14.508658,387,16.118096,2.309401e+06,NaN
4,002a68644,14400000.0,2000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,15.866190,14.953948,15.201805,373,18.699403,1.154503e+07,4.598892


In [32]:
train.replace(np.nan,0, inplace=True)
test.replace(np.nan,0, inplace=True)

## Test Improvement

In [29]:
X=train[train.columns[2:]]
y=np.log1p(train['target'])
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return abs(np.sqrt(np.mean((y_pred - y_true)**2))) 

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'gamma',
    'metric': {'rmse'},
    'num_leaves': 50,
    'learning_rate': 0.01,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 4,
    'max_depth': -1,
    'reg_alpha': 0.3,
    'reg_lambda': 0.1,
    'min_child_weight': 10,
    'zero_as_missing': True,
    'verbose': 1
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Start predicting...')
# predict
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', root_mean_squared_error(y_test, y_pred))

Start training...
[1]	valid_0's rmse: 1.72602
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's rmse: 1.71994
[3]	valid_0's rmse: 1.70438
[4]	valid_0's rmse: 1.68898
[5]	valid_0's rmse: 1.6827
[6]	valid_0's rmse: 1.66799
[7]	valid_0's rmse: 1.65315
[8]	valid_0's rmse: 1.64727
[9]	valid_0's rmse: 1.63253
[10]	valid_0's rmse: 1.61808
[11]	valid_0's rmse: 1.60386
[12]	valid_0's rmse: 1.58997
[13]	valid_0's rmse: 1.57619
[14]	valid_0's rmse: 1.56259
[15]	valid_0's rmse: 1.54921
[16]	valid_0's rmse: 1.54405
[17]	valid_0's rmse: 1.5307
[18]	valid_0's rmse: 1.51754
[19]	valid_0's rmse: 1.51273
[20]	valid_0's rmse: 1.50006
[21]	valid_0's rmse: 1.48771
[22]	valid_0's rmse: 1.48299
[23]	valid_0's rmse: 1.47087
[24]	valid_0's rmse: 1.45894
[25]	valid_0's rmse: 1.45429
[26]	valid_0's rmse: 1.44244
[27]	valid_0's rmse: 1.43054
[28]	valid_0's rmse: 1.41894
[29]	valid_0's rmse: 1.40744
[30]	valid_0's rmse: 1.39613
[31]	valid_0's rmse: 1.39188
[32]	valid_0's rmse: 1.38761
[33]

[308]	valid_0's rmse: 0.67357
[309]	valid_0's rmse: 0.673528
[310]	valid_0's rmse: 0.673518
[311]	valid_0's rmse: 0.673236
[312]	valid_0's rmse: 0.673262
[313]	valid_0's rmse: 0.673104
[314]	valid_0's rmse: 0.672854
[315]	valid_0's rmse: 0.672443
[316]	valid_0's rmse: 0.672092
[317]	valid_0's rmse: 0.672125
[318]	valid_0's rmse: 0.671886
[319]	valid_0's rmse: 0.671629
[320]	valid_0's rmse: 0.67158
[321]	valid_0's rmse: 0.671292
[322]	valid_0's rmse: 0.671038
[323]	valid_0's rmse: 0.670782
[324]	valid_0's rmse: 0.670496
[325]	valid_0's rmse: 0.670087
[326]	valid_0's rmse: 0.670124
[327]	valid_0's rmse: 0.669848
[328]	valid_0's rmse: 0.669603
[329]	valid_0's rmse: 0.669601
[330]	valid_0's rmse: 0.669346
[331]	valid_0's rmse: 0.668999
[332]	valid_0's rmse: 0.668709
[333]	valid_0's rmse: 0.66832
[334]	valid_0's rmse: 0.668338
[335]	valid_0's rmse: 0.668054
[336]	valid_0's rmse: 0.667683
[337]	valid_0's rmse: 0.66743
[338]	valid_0's rmse: 0.667314
[339]	valid_0's rmse: 0.666942
[340]	valid_

From one of the successful kernels:

In [30]:
import time
learning_rates = [0.015,0.01]
for param in learning_rates:
    print("Learning Rate: ", param)
    modelstart= time.time()
    params["learning_rate"] = param
    # Find Optimal Parameters / Boosting Rounds
    lgb_cv = lgb.cv(
        params = params,
        train_set = lgb_train,
        num_boost_round=1000,
        stratified=False,
        nfold = 5,
        verbose_eval=200,
        seed = 23,
        early_stopping_rounds=75)

    optimal_rounds = np.argmin(lgb_cv['rmse-mean'])
    best_cv_score = min(lgb_cv['rmse-mean'])

    print("Optimal Round: {}\nOptimal Score: {} + {}".format(
        optimal_rounds,best_cv_score,lgb_cv['rmse-stdv'][optimal_rounds]))
    print("###########################################################################################")

Learning Rate:  0.015
[200]	cv_agg's rmse: 0.73944 + 0.0362683
[400]	cv_agg's rmse: 0.717533 + 0.0390627
Optimal Round: 498
Optimal Score: 0.7159211643356415 + 0.03957621054593416
###########################################################################################
Learning Rate:  0.01
[200]	cv_agg's rmse: 0.789428 + 0.0344841
[400]	cv_agg's rmse: 0.721741 + 0.0397164
[600]	cv_agg's rmse: 0.714193 + 0.0402347
[800]	cv_agg's rmse: 0.713488 + 0.0402673
Optimal Round: 739
Optimal Score: 0.713152119703968 + 0.040588511405705394
###########################################################################################


In [33]:
%%time
x_train=train[train.columns[2:]]
y_train=np.log1p(train['target'])

x_test=test[x_train.columns]

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return abs(K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)))

lgb_train = lgb.Dataset(x_train, y_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'gamma',
    'metric': {'rmse'},
    'num_leaves': 50,
    'learning_rate': 0.01,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 4,
    'max_depth': -1,
    'reg_alpha': 0.3,
    'reg_lambda': 0.1,
    'min_child_weight': 10,
    'zero_as_missing': True,
    'verbose': 0
}

# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10000)

y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)

Wall time: 1min 26s


In [34]:
predictions=pd.DataFrame({'ID':test['ID'],'target':np.expm1(y_pred)})
print(predictions.head())
predictions.to_csv('pred_lgbmleak.csv',index=False)

          ID        target
0  000137c73  1.576021e+06
1  00021489f  2.404527e+06
2  0004d7953  2.769994e+06
3  00056a333  8.087247e+06
4  00056d8eb  2.404527e+06
